## Setup

In [1]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm import tqdm
from transformer_lens import HookedTransformer
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal
from transformer_lens.utils import test_prompt
import pickle
from ipywidgets import interact, IntSlider, SelectionSlider
from transformer_lens.utils import test_prompt

import plotly.graph_objects as go

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import (
    AutoEncoderConfig, 
    get_all_activating_test_prompts, 
    eval_direction_tokens_global, 
    get_encode_activations_hook, 
    get_activations, 
    get_acts, 
    load_encoder, 
    eval_ablation_token_rank, 
    get_direction_ablation_hook, 
    get_top_activating_examples_for_direction, 
    evaluate_direction_ablation_single_prompt,
    eval_encoder_reconstruction_single_position,
    get_top_direction_ablation_df,
    get_mean_component_wise_mlp,
    get_custom_forward_hook
)
import utils.haystack_utils as haystack_utils
from utils.plotting_utils import line, multiple_line
%reload_ext autoreload
%autoreload 2

In [2]:
autoencoder_directions = {}

In [3]:
model_name = "tiny-stories-2L-33M"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)
model.set_use_attn_result(True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/323M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [6]:
run_names = ["54_serene_plasma", "189_giddy_water"] 
encoders = []
for run_name in run_names:
    encoder, cfg = load_encoder(run_name, model_name, model)
    cfg.run_name = run_name
    print(cfg.run_name, cfg.layer, cfg.l1_coeff)
    encoders.append((encoder, cfg))

54_serene_plasma 0 0.0003
189_giddy_water 1 [0.0001, 0.00015]


In [7]:
prompts = load_tinystories_validation_prompts()

(INFO) 02:31:41: Loaded 21990 TinyStories validation prompts


In [8]:
max_activation_data = {}
for encoder, cfg in encoders:
    run_name = cfg.run_name
    max_activations, max_activation_token_indices = get_activations(encoder, cfg, run_name, prompts, model)
    max_activation_data[run_name] = {
        "max_activations": max_activations.cpu(),
        "max_activation_token_indices": max_activation_token_indices.cpu()
    }

  0%|          | 0/21990 [00:00<?, ?it/s]

100%|██████████| 21990/21990 [03:47<00:00, 96.81it/s] 


Active directions on validation data: 16384 out of 16384


100%|██████████| 21990/21990 [04:41<00:00, 78.05it/s]


Active directions on validation data: 16383 out of 16384


In [9]:
def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [10]:
first_encoder, first_encoder_cfg = encoders[0]
second_encoder, second_encoder_cfg = encoders[1]

first_encoder_max_activations = max_activation_data[first_encoder_cfg.run_name]["max_activations"]
first_encoder_max_activation_token_indices = max_activation_data[first_encoder_cfg.run_name]["max_activation_token_indices"]
second_encoder_max_activations = max_activation_data[second_encoder_cfg.run_name]["max_activations"]
second_encoder_max_activation_token_indices = max_activation_data[second_encoder_cfg.run_name]["max_activation_token_indices"]

In [11]:
def get_common_tinystories_tokens(prompts, model: HookedTransformer, min_occurrences=100):
    occurrences = torch.zeros(model.cfg.d_vocab, dtype=torch.int32).cuda()
    for prompt in prompts: 
        tokens = model.to_tokens(prompt).flatten()
        occurrences = occurrences.index_add(0, tokens, torch.ones_like(tokens, dtype=torch.int32))
    common_tokens = torch.argwhere(occurrences > min_occurrences).flatten()
    rare_tokens = torch.argwhere(occurrences <= min_occurrences).flatten()
    return occurrences, common_tokens, rare_tokens

occurrences, common_tokens, rare_tokens = get_common_tinystories_tokens(prompts, model)
print(len(common_tokens), len(rare_tokens))

2552 47705


In [12]:
# Filter test prompts following 'said, " [...] ."' pattern
# '."' '?"' and '!"' are single tokens

answer_token = model.to_single_token(".\"")
test_prompts = []
for prompt in tqdm(prompts):
    if "said, \"" in prompt:
        start_index = prompt.index("said, \"") + 7
        end_index = prompt.find(".\"", start_index)
        # Exclude long prompts - model performance degrades
        if (end_index != -1) and (end_index < 1600):
            subprompt = prompt[:end_index+2]
            tokens = model.to_tokens(subprompt)
            last_token = model.to_single_str_token(tokens[0, -1].item())
            if (subprompt[-2:] == ".\"") and (last_token == ".\""):
                test_prompts.append(subprompt)
print(len(test_prompts))

100%|██████████| 21990/21990 [00:02<00:00, 9481.56it/s] 

4500


## Model DLA

In [13]:
def DLA(prompts: list[str], model: HookedTransformer, pos=-1) -> tuple[Float[Tensor, "component"], list[str]]:
    logit_attributions = []
    for prompt in tqdm(prompts):
        tokens = model.to_tokens(prompt)
        answers = tokens[:, 1:]
        tokens = tokens[:, :-1]
        answer_residual_directions = model.tokens_to_residual_directions(answers)[0, pos]  # [batch pos d_model]
        _, cache = model.run_with_cache(tokens)
        accumulated_residual, labels = cache.get_full_resid_decomposition(layer=-1, pos_slice=pos, return_labels=True, expand_neurons=False)
        scaled_residual_stack = cache.apply_ln_to_stack(accumulated_residual, layer = -1, pos_slice=pos).squeeze(1)
        logit_attribution = einops.einsum(scaled_residual_stack, answer_residual_directions, "component d_model, d_model -> component") # / answers.shape[0]
        logit_attributions.append(logit_attribution)
    
    logit_attributions = torch.stack(logit_attributions).mean(0)
    return logit_attributions, labels

In [14]:
# DLA of '."', find relevant MLPs if they exist, check encoders of those MLPs for relevant directions
# DLA looks pretty different for different prompts
# Relevant boosting heads:
# Relevant boosting MLP: 
# Relevant deboosting heads: 
# Relevant deboosting MLP:
dlas, labels = DLA(test_prompts, model, pos=-1)
print(dlas.shape)

100%|██████████| 4500/4500 [00:39<00:00, 112.93it/s]

torch.Size([37])


In [15]:
l1_attention_dla = sum([dla.item() for dla, label in zip(dlas, labels) if "L1H" in label])
print(l1_attention_dla)

6.00066873896867


In [16]:
line(dlas.cpu().numpy(), xticks=labels, width=1200, title="Closing quotation DLA", show_legend=False)

## Ablate L1 Directions

In [17]:
activating_test_prompts_l1 = get_all_activating_test_prompts(test_prompts, second_encoder, model, second_encoder_cfg, active_threshold=0.1)

 59%|█████▉    | 2666/4500 [00:30<00:20, 87.85it/s]

100%|██████████| 4500/4500 [00:51<00:00, 87.99it/s]


In [18]:
original_loss, ablated_loss, encoded_loss = eval_encoder_reconstruction_single_position(test_prompts[200:500], second_encoder, model, second_encoder_cfg)
print(f"Original loss: {original_loss:.3f}, ablated loss: {ablated_loss:.3f}, encoded loss: {encoded_loss:.3f}")
print(f"Loss recovered: {(ablated_loss - encoded_loss)/(ablated_loss - original_loss):.4f}")

  3%|▎         | 8/300 [00:00<00:08, 36.29it/s]

100%|██████████| 300/300 [00:08<00:00, 35.66it/s]

Original loss: 0.968, ablated loss: 1.832, encoded loss: 1.045
Loss recovered: 0.9109


In [19]:
df_l1, loss_increases_l1 = get_top_direction_ablation_df(activating_test_prompts_l1, test_prompts, model, second_encoder, second_encoder_cfg, second_encoder_max_activations)
df_l1.head(10)

Number of directions with mean activation > 0: 34


100%|██████████| 34/34 [02:56<00:00,  5.18s/it]


,Direction,Loss increase,Loss increase (encoded),Mean activation,Percentage activation
0,82,0.073210,-0.012924,5.005048,0.026
1,11865,0.150870,0.167747,3.859326,0.030
2,8093,0.496260,0.477907,3.815932,0.471
3,14643,0.177023,0.144021,3.262734,0.021
4,15796,0.136471,0.166351,2.788814,0.069
5,6011,0.195737,0.392007,2.756625,0.119
6,5000,0.007704,0.106521,2.548186,0.020
7,5914,0.049325,0.032349,2.516097,0.020
8,794,0.312786,0.395805,2.493319,0.411
9,11157,0.028837,-0.086961,2.467405,0.023


In [20]:
directions = df_l1["Direction"].tolist()
# Preparing DataFrame
df = pd.DataFrame([(direction, value) for direction, losses in zip(directions, loss_increases_l1) for value in losses],
                  columns=['Direction', 'Loss Increase'])

# Creating histogram
fig = px.histogram(df, x="Loss Increase", color="Direction", barmode='overlay')

# Updating labels and title
fig.update_layout(
    xaxis_title='Loss Increase',
    yaxis_title='Count',
    title='Loss Increase Histogram by Direction'
)

fig.show()

In [21]:
df = df_l1.sort_values("Loss increase", ascending=False)
top_directions = df["Direction"].tolist()
directions = top_directions[:3]
print(f"The top 3 directions are {directions}")
df.head(10)

The top 3 directions are [8093, 794, 6011]


,Direction,Loss increase,Loss increase (encoded),Mean activation,Percentage activation
2,8093,0.496260,0.477907,3.815932,0.471
8,794,0.312786,0.395805,2.493319,0.411
5,6011,0.195737,0.392007,2.756625,0.119
10,13657,0.181290,0.260132,2.457184,0.020
3,14643,0.177023,0.144021,3.262734,0.021
18,10665,0.163724,0.235448,2.031421,0.027
1,11865,0.150870,0.167747,3.859326,0.030
4,15796,0.136471,0.166351,2.788814,0.069
17,8594,0.103299,0.143853,2.041233,0.037
32,6258,0.100347,0.191090,1.181257,0.040


## Between encoder similarity

In [22]:
top_directions = df_l1.head(9)["Direction"].tolist()
top_directions_w_enc = second_encoder.W_enc[:, top_directions].cpu().numpy()
top_directions_w_dec = second_encoder.W_dec[top_directions, :].cpu().numpy()

In [23]:
# Cosine sims between top_directions_w_enc
cosine_sims = np.zeros((len(top_directions), len(top_directions)))
for i in range(len(top_directions)):
    for j in range(len(top_directions)):
        cosine_sims[i, j] = np.dot(top_directions_w_enc[i], top_directions_w_enc[j]) / (np.linalg.norm(top_directions_w_enc[i]) * np.linalg.norm(top_directions_w_enc[j]))
print(np.round(cosine_sims,2))

[[ 1.    0.09 -0.35  0.36  0.29  0.21  0.01  0.01  0.44]
 [ 0.09  1.   -0.2   0.25  0.09 -0.53 -0.38  0.49  0.29]
 [-0.35 -0.2   1.   -0.41  0.22  0.31  0.02 -0.13 -0.23]
 [ 0.36  0.25 -0.41  1.   -0.16 -0.25 -0.69  0.51 -0.01]
 [ 0.29  0.09  0.22 -0.16  1.    0.47  0.36  0.05 -0.15]
 [ 0.21 -0.53  0.31 -0.25  0.47  1.    0.57 -0.35 -0.31]
 [ 0.01 -0.38  0.02 -0.69  0.36  0.57  1.   -0.6  -0.11]
 [ 0.01  0.49 -0.13  0.51  0.05 -0.35 -0.6   1.   -0.13]
 [ 0.44  0.29 -0.23 -0.01 -0.15 -0.31 -0.11 -0.13  1.  ]]


In [24]:
# Cosine sims between top_directions_w_dec
cosine_sims = np.zeros((len(top_directions), len(top_directions)))
for i in range(len(top_directions)):
    for j in range(len(top_directions)):
        cosine_sims[i, j] = np.dot(top_directions_w_dec[i], top_directions_w_dec[j]) / (np.linalg.norm(top_directions_w_dec[i]) * np.linalg.norm(top_directions_w_dec[j]))
print(np.round(cosine_sims,2))

[[1.   0.16 0.17 0.27 0.21 0.16 0.14 0.04 0.21]
 [0.16 1.   0.13 0.02 0.15 0.16 0.01 0.05 0.27]
 [0.17 0.13 1.   0.13 0.45 0.34 0.08 0.18 0.43]
 [0.27 0.02 0.13 1.   0.26 0.12 0.06 0.05 0.16]
 [0.21 0.15 0.45 0.26 1.   0.42 0.04 0.12 0.55]
 [0.16 0.16 0.34 0.12 0.42 1.   0.03 0.08 0.53]
 [0.14 0.01 0.08 0.06 0.04 0.03 1.   0.17 0.1 ]
 [0.04 0.05 0.18 0.05 0.12 0.08 0.17 1.   0.15]
 [0.21 0.27 0.43 0.16 0.55 0.53 0.1  0.15 1.  ]]


In [25]:
autoencoder_directions[second_encoder_cfg.run_name] = {
    "df": df_l1,
    "top_directions": top_directions,
    "top_directions_w_enc": top_directions_w_enc,
    "top_directions_w_dec": top_directions_w_dec,
}

In [26]:
all_mean_directions = []
for key in autoencoder_directions.keys():
    df = autoencoder_directions[key]["df"]
    num_directions = len(df[df["Loss increase"]> 0.05])
    print(f"{key}: {num_directions} directions")
    directions = autoencoder_directions[key]["top_directions_w_dec"][:num_directions]
    mean_direction = directions.mean(0)
    all_mean_directions.append(mean_direction)
all_mean_directions = np.stack(all_mean_directions, 1)
print(all_mean_directions.shape)


189_giddy_water: 19 directions
(4096, 1)


In [27]:
# Cosine sim between directions
cosine_sim = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        cosine_sim[i, j] = np.dot(all_mean_directions[:, i], all_mean_directions[:, j]) / (np.linalg.norm(all_mean_directions[:, i]) * np.linalg.norm(all_mean_directions[:, j]))

cosine_sim

IndexError: index 1 is out of bounds for axis 1 with size 1

## MLP remainder term

In [ ]:
# Compute MLP remainder
# DLA of MLP compared to DLA of reconstructed MLP
# Mean activations over a bunch of test prompts
# Mean reconstructions over a bunch of test prompts
# Apply LN final layer to both
# Compute DLA of both

dlas = []
hook_name = f"blocks.{second_encoder_cfg.layer}.{second_encoder_cfg.act_name}"
for prompt in test_prompts[:200]:
    _, cache = model.run_with_cache(prompt)
    mlp_acts = cache[hook_name][0, -2]
    
    _, x_reconstruct, _, _, _ = second_encoder(mlp_acts)
    
    mlp_res = mlp_acts @ model.W_out[second_encoder_cfg.layer]
    reconstruct_res = x_reconstruct @ model.W_out[second_encoder_cfg.layer]
    remainder = (mlp_res - reconstruct_res) 

    stacked_residual = torch.stack([mlp_res, reconstruct_res, remainder])
    stacked_residual = cache.apply_ln_to_stack(stacked_residual, pos_slice=-2)

    dla = stacked_residual @ model.W_U[:, answer_token]

    dlas.append(dla)

dla = torch.stack(dlas).mean(0)
print(dla.shape)


torch.Size([3])


In [ ]:
line(dla.tolist(), xticks=["MLP", "Reconstruct", "Remainder"], show_legend=False, title="Last MLP layer DLA for closing quotation prompts")

## Direction DFA 

In [28]:
# Check neuron basis for directions
# Cosine sims of direction encoder weights
# Check if directions activate together or on separate examples
# Compare set of top 100 contributing neurons for each direction for overlap
# DFA for directions

In [29]:
directions = df["Direction"][:1].tolist()
directions

[82]

In [30]:
def direction_dla(test_prompts, direction, activating_test_prompts, encoder, encoder_cfg, model, mean_mlp_decomp= None, pos=-2):
    activating_test_prompt_indices = torch.argwhere(activating_test_prompts[:, direction] > 0).flatten().tolist()

    direction_weight = encoder.W_enc[:, direction]
    dlas = []
    for prompt_index in activating_test_prompt_indices:
        prompt = test_prompts[prompt_index]
        _, cache = model.run_with_cache(prompt)

        decomposition, labels = cache.get_full_resid_decomposition(encoder_cfg.layer, mlp_input=True, apply_ln=True, return_labels=True, expand_neurons=False, pos_slice=pos)
        decomposition = decomposition.squeeze(1)

        # Account for GELU in DLA by setting neuron contributions to 0 if they are not activated
        mlp_wise_decomposition = einops.einsum(decomposition, model.W_in[encoder_cfg.layer], "component d_res, d_res d_mlp -> component d_mlp")
        mlp_activations = cache[f"blocks.{encoder_cfg.layer}.mlp.hook_post"][0, pos, :]
        zeroed_neurons = torch.argwhere(mlp_activations <= 0).flatten()
        if mean_mlp_decomp is not None:
            mlp_wise_decomposition[:, zeroed_neurons] = mean_mlp_decomp[:, zeroed_neurons]
        else:
            mlp_wise_decomposition[:, zeroed_neurons] = 0

        dla = einops.einsum(mlp_wise_decomposition, direction_weight, "component d_mlp, d_mlp -> component")
        dlas.append(dla)
    dla = torch.stack(dlas).mean(0).tolist()
    return dla, labels

In [31]:
dlas = []
mean_mlp_decomp = get_mean_component_wise_mlp(prompts[:100], model, second_encoder_cfg)
for direction in directions:
    dla, labels = direction_dla(test_prompts, direction, activating_test_prompts_l1, second_encoder, second_encoder_cfg, model, mean_mlp_decomp=mean_mlp_decomp)
    dlas.append(dla)
multiple_line(dlas, directions, xticks=labels, title="DFA for top directions (mean ablation of inactive MLP neurons)", width=1000)

In [32]:
dlas = []
for direction in directions:
    dla, labels = direction_dla(test_prompts, direction, activating_test_prompts_l1, second_encoder, second_encoder_cfg, model, mean_mlp_decomp=None)
    dlas.append(dla)
multiple_line(dlas, directions, xticks=labels, title="DFA for top directions (zero ablation of inactive MLP neurons)", width=1000)

In [33]:
# Iterate through components, ablate, check ablation loss and top direction activation

def zero_ablate_component_hook(value, hook):
    value[:, -2] = 0
    return value

def mean_ablate_component_hook(value, hook):
    value[:, -2] = component_activations[hook.name]
    return value


components = ['original', 'hook_embed', 'hook_pos_embed', 'blocks.0.hook_attn_out', 'blocks.0.hook_mlp_out', 'blocks.1.hook_attn_out', 'blocks.1.hook_mlp_out']
component_activations = {c:[] for c in components}

for prompt in prompts[:1000]:
    tokens = model.to_tokens(prompt, prepend_bos=False)

    _, cache = model.run_with_cache(tokens)

    for component in components[1:]:
        activation = cache[component].mean((0,1))
        component_activations[component].append(activation)

for component in components[1:]:
    component_activations[component] = torch.stack(component_activations[component]).mean(0)

In [ ]:
def get_direction_ablation_df(prompts: list[str], directions: list[int]):
    data = []
    for component in tqdm(components):
        losses = []
        direction_acts = []
        for prompt in prompts:
            tokens = model.to_tokens(prompt, prepend_bos=False)
            if component == "original":
                loss = model(tokens, return_type="loss", loss_per_token=True)[0, -1].item()
                acts = get_acts(tokens, model, second_encoder, second_encoder_cfg)[-2][directions]
            else:
                with model.hooks([(component, mean_ablate_component_hook)]):
                    loss = model(tokens, return_type="loss", loss_per_token=True)[0, -1].item()
                    acts = get_acts(tokens, model, second_encoder, second_encoder_cfg)[-2][directions]
            direction_acts.append(acts)
            losses.append(loss)
        direction_acts = torch.stack(direction_acts)
        mean_direction_acts = direction_acts.mean(0).tolist()
        quantile_acts = torch.quantile(direction_acts, 0.90, dim=0).tolist()
        quantile_loss = np.quantile(losses, 0.90)
        data.append([component, np.mean(loss), quantile_loss] + mean_direction_acts + quantile_acts)
    df = pd.DataFrame(data, columns=["Component", "Loss (mean)", "Loss (90th)"] + [f"Direction {i} (mean)" for i in directions] + [f"Direction {i} (90th)" for i in directions])
    return df

In [ ]:
active_test_prompts = get_active_test_prompts(test_prompts, activating_test_prompts_l1, direction)
df_tmp = get_direction_ablation_df(active_test_prompts, [direction])
df_tmp

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [05:35<00:00, 47.96s/it]


,Component,Loss (mean),Loss (90th),Direction 8093 (mean),Direction 8093 (90th)
0,original,1.218927,2.220135,2.722246,5.246456
1,hook_embed,2.693964,3.325236,1.369507,2.634907
2,hook_pos_embed,1.773568,3.644671,2.199819,4.117949
3,blocks.0.hook_attn_out,5.516525,7.925878,0.417901,1.173569
4,blocks.0.hook_mlp_out,0.969827,2.774750,2.297937,4.085497
5,blocks.1.hook_attn_out,6.058527,10.280762,0.075156,0.303328
6,blocks.1.hook_mlp_out,2.086442,4.122573,2.722246,5.246456


In [ ]:
cache

ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.hook_result', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.attn.hook_result', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.ho

In [ ]:
cache['blocks.1.attn.hook_result'].shape

torch.Size([1, 102, 16, 1024])

In [ ]:
# Iterate over attention heads

def get_mean_ablate_attention_head_hook(layer, head):
    def ablate_hook(value, hook):
        value[:, -2, head] = cache[hook.name][:, -2, ]
        return value

In [ ]:
data = []
for component in tqdm(components):
    losses = []
    direction_acts = []
    for prompt in test_prompts[:500]:
        tokens = model.to_tokens(prompt, prepend_bos=False)
        if component == "original":
            loss = model(tokens, return_type="loss", loss_per_token=True)[0, -1].item()
            acts = get_acts(tokens, model, second_encoder, second_encoder_cfg)[-2][directions]
        else:
            with model.hooks([(component, mean_ablate_component_hook)]):
                loss = model(tokens, return_type="loss", loss_per_token=True)[0, -1].item()
                acts = get_acts(tokens, model, second_encoder, second_encoder_cfg)[-2][directions]
        direction_acts.append(acts)
        losses.append(loss)
    direction_acts = torch.stack(direction_acts)
    mean_direction_acts = direction_acts.mean(0).tolist()
    quantile_acts = torch.quantile(direction_acts, 0.90, dim=0).tolist()
    quantile_loss = np.quantile(losses, 0.90)
    data.append([component, np.mean(loss), quantile_loss] + mean_direction_acts + quantile_acts)
df = pd.DataFrame(data, columns=["Component", "Loss (mean)", "Loss (90th)"] + [f"Direction {i} (mean)" for i in directions] + [f"Direction {i} (90th)" for i in directions])
df.head(10)

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [01:06<00:00,  9.48s/it]


,Component,Loss (mean),Loss (90th),Direction 8093 (mean),Direction 8093 (90th)
0,original,0.044562,2.248487,1.713200,4.891146
1,hook_embed,0.853199,3.388864,0.883988,2.486666
2,hook_pos_embed,0.448618,3.497252,1.393478,3.871100
3,blocks.0.hook_attn_out,3.684969,7.946192,0.259612,0.984259
4,blocks.0.hook_mlp_out,0.485114,2.667844,1.453291,3.825793
5,blocks.1.hook_attn_out,8.031485,10.159293,0.050195,0.113840
6,blocks.1.hook_mlp_out,0.987445,3.834849,1.713200,4.891146


In [ ]:
px.bar(df, x="Component", y="Loss", title="Mean ablation loss of components", width=700)

In [ ]:
# Zero ablate MLP0, check direction activations?
def zero_ablate_hook(value, hook):
    value[0, -2] = 0
    return value

zero_ablate_mlp0_hooks = [(first_encoder_cfg.encoder_hook_point, zero_ablate_hook)]

top_direction_activations = []
for prompt in test_prompts[550:600]:
    with model.hooks(zero_ablate_mlp0_hooks):
        acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-2][directions]
        top_direction_activations.append(acts)

top_direction_activations = torch.stack(top_direction_activations)
print(top_direction_activations.mean(0))

tensor([5.0077], device='cuda:0')


In [ ]:
losses = []
length = [len(prompt) for prompt in test_prompts]
for prompt in test_prompts:
    loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    losses.append(loss)

In [ ]:
line(losses)

In [ ]:
px.scatter(x=length, y=losses)

In [ ]:
# Compare MLP0 patched direction activations to original direction activations
print(df_l1["Mean activation"][:3])

0    3.813782
3    2.493319
2    2.756625
Name: Mean activation, dtype: float64


In [ ]:
def get_active_test_prompts(test_prompts: list[str], activating_test_prompts: Tensor, direction: int):
    assert len(test_prompts) == activating_test_prompts.shape[0]
    activating_prompt_indices = activating_test_prompts[:, direction].nonzero().flatten().tolist()
    activating_prompts = [test_prompts[i] for i in activating_prompt_indices]
    return activating_prompts

In [ ]:
direction

8093

In [ ]:
import circuitsvis as cv

prompt = get_active_test_prompts(test_prompts, activating_test_prompts_l1, direction)[504]
_, cache = model.run_with_cache(prompt)
pattern = cache["pattern", 1][:, [11, 13]].squeeze(0)
print(pattern.shape)

# display(cv.attention.attention_patterns(
#         attention = pattern.cpu(),
#         tokens = model.to_str_tokens(prompt),
#         attention_head_names = ["L1H11", "L1H13"],
#     ))

torch.Size([2, 102, 102])


In [ ]:
# Correlation between activating test prompts
direction_activations = activating_test_prompts_l1[:, directions].to(torch.int32)
correlation_matrix = torch.corrcoef(direction_activations.T)
print(correlation_matrix.triu(1))

## L0 Directions

In [ ]:
activating_test_prompts_l0 = get_all_activating_test_prompts(test_prompts, first_encoder, model, first_encoder_cfg, active_threshold=0.05)

100%|██████████| 4554/4554 [01:16<00:00, 59.87it/s]


In [ ]:
original_loss, ablated_loss, encoded_loss = eval_encoder_reconstruction_single_position(test_prompts[:200], first_encoder, model, first_encoder_cfg)
print(f"Original loss: {original_loss:.3f}, ablated loss: {ablated_loss:.3f}, encoded loss: {encoded_loss:.3f}")
print(f"Loss recovered: {(ablated_loss - encoded_loss)/(ablated_loss - original_loss):.4f}")

100%|██████████| 200/200 [00:04<00:00, 44.09it/s]

Original loss: 0.844, ablated loss: 2.248, encoded loss: 0.835
Loss recovered: 1.0060


In [ ]:
df_l0 = get_top_direction_ablation_df(activating_test_prompts_l0, test_prompts, model, first_encoder, first_encoder_cfg, first_encoder_max_activations)
df_l0.head(10)

Number of directions with mean activation > 0: 17


100%|██████████| 17/17 [02:16<00:00,  8.05s/it]

The top 3 directions are [1154, 4776, 5889]


,Direction,Loss increase,Loss increase (encoded),Mean activation,Percentage activation
15,1154,0.028411,0.048419,0.642378,0.055
7,4776,0.017387,0.060828,0.869907,0.055
12,5889,0.014969,-0.004010,0.784309,0.340
10,2952,0.008873,0.015629,0.801824,0.053
6,9769,0.001664,-0.005776,0.870337,0.103
8,7944,0.001470,-0.021325,0.860028,0.071
14,4260,-0.001700,0.016599,0.729655,0.341
13,4970,-0.005092,0.016114,0.763923,0.058
9,4063,-0.005863,-0.040843,0.839603,0.120
11,2652,-0.007915,-0.019286,0.784318,0.110


In [ ]:
l0_directions = df_l0["Direction"][:4].tolist()

In [ ]:
from utils.autoencoder_utils import get_custom_forward_hook
hook = get_custom_forward_hook(first_encoder, l0_directions, 0, first_encoder_cfg, pos=-2)

loss_increases_encoded_ablation = []
for prompt in test_prompts[:200]:
    tokens = model.to_tokens(prompt, prepend_bos=False)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, encoder, model, l0_directions, cfg, pos=pos)
    with model.hooks(hook):
        encoded_ablated_loss = model(tokens, return_type="loss", loss_per_token=True)[0, -1].item()
    loss_increases_encoded_ablation.append(encoded_ablated_loss - original_loss)
print(np.mean(loss_increases_encoded_ablation))

0.019882183712907134


In [ ]:
# Ablate multiple l0 directions
loss_increases = []
for prompt in test_prompts[:200]:
    tokens = model.to_tokens(prompt)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, first_encoder, model, l0_directions, first_encoder_cfg, pos=pos)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

0.015580674074590206 0.039593848407671516


In [ ]:
px.histogram(loss_increases, title="Loss increase from ablation of multiple L0 directions")

## Ablate set of important L1 directions

In [ ]:
#directions = [3373, 15330, 2594, 8842, 7447]
directions = df_l1["Direction"][:5].tolist()
print(len(directions), directions)

5 [8093, 794, 6011, 15796, 2863]


In [ ]:
# On positions which close quotation
loss_increases = []
for prompt in test_prompts[:200]:
    tokens = model.to_tokens(prompt)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, second_encoder, model, directions, second_encoder_cfg, pos=pos)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

0.5854962168075144 0.5203302657134952


In [ ]:
answer_token = ".\""
n = 200
data = []
for test_prompt_index, prompt in tqdm(enumerate(test_prompts[:n]), total=n):
    answer_logprob, ablated_answer_logprob, answer_logit, ablated_answer_logit, answer_rank, ablated_answer_rank = eval_ablation_token_rank(prompt, second_encoder, model, directions, second_encoder_cfg, answer_token, pos=-2)
    data.append([test_prompt_index, answer_logprob, ablated_answer_logprob, answer_logit, ablated_answer_logit, answer_rank, ablated_answer_rank])
ablation_df = pd.DataFrame(data, columns=["Prompt index", "Answer logprob", "Ablated answer logprob", "Answer logit", "Ablated answer logit", "Answer rank", "Ablated answer rank"])
# Calculate differences between original and ablated measures
ablation_df['Logprob Difference'] = ablation_df['Answer logprob'] - ablation_df['Ablated answer logprob']
ablation_df['Logit Difference'] = ablation_df['Answer logit'] - ablation_df['Ablated answer logit']
ablation_df['Rank Difference'] = ablation_df['Answer rank'] - ablation_df['Ablated answer rank']

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [00:03<00:00, 64.12it/s]


In [ ]:
def plot_ablation_losses(ablation_df):
    # Calculate means, standard deviations, and standard errors for differences
    means_diff = ablation_df[['Logprob Difference', 'Logit Difference', 'Rank Difference']].mean()
    stds_diff = ablation_df[['Logprob Difference', 'Logit Difference', 'Rank Difference']].std()

    # Standard Errors
    stderrs_diff = stds_diff / np.sqrt(n)

    # 95% Confidence Intervals for Differences
    confidence_interval_diff = 1.96 * stderrs_diff

    # Lower and upper bounds for differences
    lower_bounds_diff = means_diff - confidence_interval_diff
    upper_bounds_diff = means_diff + confidence_interval_diff

    # Create a bar plot for differences
    fig = go.Figure()
    metrics = ['Logprob Difference', 'Logit Difference', 'Rank Difference']

    for metric in metrics:
        fig.add_trace(go.Bar(
            x=[metric],
            y=[means_diff[metric]],
            error_y=dict(
                type='data',  # Represents actual data points
                array=[upper_bounds_diff[metric] - means_diff[metric]],
                arrayminus=[means_diff[metric] - lower_bounds_diff[metric]]
            ),
            name=metric
        ))

    fig.update_layout(
        title="Original - Ablated Measures",
        xaxis_title="Metric",
        yaxis_title="Difference",
        barmode='group',
        width=700
    )

    fig.show()


In [ ]:
plot_ablation_losses(ablation_df)

In [ ]:
# Baseline check: mean ablate MLP1
hook_name = f"blocks.{second_encoder_cfg.layer}.{second_encoder_cfg.act_name}"
mean_activation = []
for prompt in prompts[:10]:
    _, cache = model.run_with_cache(prompt)
    activations = cache[hook_name][0].mean(0)
    mean_activation.append(activations)
mean_activation = torch.stack(mean_activation).mean(0)

In [ ]:
def mean_ablation_hook(value, hook):
    value[0, -2] = mean_activation 
    return value

In [ ]:
loss_increases = []
answer_token_index = model.to_single_token(answer_token)
pos = -2
data = []
for prompt in test_prompts[:100]:
    logits = model(prompt, return_type="logits")[0, pos]
    answer_rank = (logits > logits[answer_token_index]).sum().item()
    answer_logprob = logits.log_softmax(dim=-1)[answer_token_index].item()
    answer_logit = logits[answer_token_index].item()
    with model.hooks(fwd_hooks=[(hook_name, mean_ablation_hook)]):
        ablated_logits = model(prompt, return_type="logits")[0, pos]
        ablated_answer_rank = (ablated_logits > ablated_logits[answer_token_index]).sum().item()
        ablated_answer_logprob = ablated_logits.log_softmax(dim=-1)[answer_token_index].item()
        ablated_answer_logit = ablated_logits[answer_token_index].item()
    data.append([test_prompt_index, answer_logprob, ablated_answer_logprob, answer_logit, ablated_answer_logit, answer_rank, ablated_answer_rank])
ablation_df = pd.DataFrame(data, columns=["Prompt index", "Answer logprob", "Ablated answer logprob", "Answer logit", "Ablated answer logit", "Answer rank", "Ablated answer rank"])
# Calculate differences between original and ablated measures
ablation_df['Logprob Difference'] = ablation_df['Answer logprob'] - ablation_df['Ablated answer logprob']
ablation_df['Logit Difference'] = ablation_df['Answer logit'] - ablation_df['Ablated answer logit']
ablation_df['Rank Difference'] = ablation_df['Answer rank'] - ablation_df['Ablated answer rank']
plot_ablation_losses(ablation_df)

## Ablate nullspace of important directions

In [ ]:
from scipy.linalg import null_space

def get_direct_sum_complement(basis):
    basis_matrix = np.column_stack(basis)
    return null_space(basis_matrix.T)
 
print(type(df_l1["Direction"][:5]))
subspace_basis = np.array(df_l1["Direction"][:5]) # Basis for a subspace in R^d_hidden

complement = get_direct_sum_complement(subspace_basis)
print(complement)

TypeError: Cannot construct a dtype from an array

## Loss recovered analysis

In [ ]:
# Baseline check: mean ablate MLP1
hook_name = f"blocks.{second_encoder_cfg.layer}.{second_encoder_cfg.act_name}"

mean_activation = []
for prompt in prompts[:20]:
    _, cache = model.run_with_cache(prompt)
    activations = cache[hook_name][0].mean(0)
    mean_activation.append(activations)
mean_activation = torch.stack(mean_activation).mean(0)

def mean_ablation_hook(value, hook):
    value[0, -2] = mean_activation 
    return value

def zero_ablation_hook(value, hook):
    value[0, -2] = 0
    return value

def encode_activations_hook(value, hook):
    _, x_reconstruct, acts, _, _ = second_encoder(value[:, -2])
    value[:, -2] = x_reconstruct
    return value

ablate_top_directions_hook = [(hook_name, get_direction_ablation_hook(second_encoder, directions, -2))]
encode_mlp_hook = [(hook_name, encode_activations_hook)]

In [ ]:
# Zero ablate MLP
# ((zero_abl_loss - recons_loss)/(zero_abl_loss - loss))
# Mean ablate MLP
# Enable only top 2 directions
# Enable full autoencoder

losses = []
mean_abl_losses = []
zero_abl_losses = []
top_abl_losses = []
recons_losses = []

for prompt in tqdm(test_prompts):
    loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=[(hook_name, mean_ablation_hook)]):
        mean_abl_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=[(hook_name, zero_ablation_hook)]):
        zero_abl_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=ablate_top_directions_hook):
        top_abl_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    with model.hooks(fwd_hooks=encode_mlp_hook):
        recons_loss = model(prompt, return_type="loss", loss_per_token=True)[0, -1].item()
    losses.append(loss)
    mean_abl_losses.append(mean_abl_loss)
    zero_abl_losses.append(zero_abl_loss)
    top_abl_losses.append(top_abl_loss)
    recons_losses.append(recons_loss)


  0%|          | 0/4554 [00:00<?, ?it/s]

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Calculate means and standard deviations
means = [
    np.mean(losses),
    np.mean(mean_abl_losses),
    np.mean(zero_abl_losses),
    np.mean(top_abl_losses),
    np.mean(recons_losses)
]

def standard_error(data):
    return np.std(data) / np.sqrt(len(data))

std_errors = [
    standard_error(losses),
    standard_error(mean_abl_losses),
    standard_error(zero_abl_losses),
    standard_error(top_abl_losses),
    standard_error(recons_losses)
]

ci_95 = [se * 1.96 for se in std_errors]

labels = ['Original loss', 'MLP mean ablation loss', 'MLP zero ablation loss', f'Top {len(directions)} SAE directions ablation loss', 'SAE reconstruction loss']
fig = go.Figure(data=[
    go.Bar(
        x=labels,
        y=means,
        error_y=dict(type='data', array=ci_95)
    )
])

fig.update_layout(
    title="Ablation loss comparison for closing quotation prompts '.\"'",
    xaxis_title="Ablation",
    yaxis_title="Loss",
    showlegend=False,
    width=600
)

fig.show()


In [ ]:
loss_recovered = (np.mean(zero_abl_losses) - np.mean(recons_losses)) / (np.mean(zero_abl_losses) - np.mean(losses))
print(loss_recovered)

0.8266587223460056


## Individual Directions

In [ ]:
direction = 8093		
print(direction)

8093


In [ ]:
from ipywidgets import interactive, IntSlider, SelectionSlider

def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

# def print_direction_example(direction, num_examples=3):
#     print_top_examples(prompts, second_encoder_max_activations, direction, second_encoder, second_encoder_cfg, num_examples)
# widget = interactive(print_direction_example, 
#              direction=SelectionSlider(options=directions, value=directions[0], description='Direction'),
#              num_examples=IntSlider(min=1, max=20, step=1, value=5))
# display(widget)

# 15xxx: activates within quotation when ending the sentence is good
# 8093: Activates always within quotation

print_top_examples(prompts, second_encoder_max_activations, direction, second_encoder, second_encoder_cfg, 5)

In [ ]:
# Filter test prompts by which prompts the direction activates on
activating_test_prompt_indices = activating_test_prompts_l1[:, direction].nonzero().flatten().tolist()
activating_test_prompts = [test_prompts[i] for i in activating_test_prompt_indices]
print(len(activating_test_prompts), "out of", activating_test_prompts_l1.shape[0])

2620 out of 4554


In [ ]:
answer_token = model.to_single_token(".\"")
quotation_tokens = ["\"", ".", "!", "?", ".\"", "!\"", "?\""]
for quotation_token in quotation_tokens:
    assert model.to_single_token(quotation_token) in common_tokens.tolist()

print(answer_token, model.to_single_str_token(answer_token))

526 ."


In [ ]:
# Logprob boosts (prompt based)
def get_direction_logit_and_logprob_boost(
    prompts: list[str],
    encoder: AutoEncoder,
    encoder_neuron,
    model: HookedTransformer,
    all_ignore: Int[Tensor, "tokens"],
    cfg: AutoEncoderConfig,
    pos: -2
):
    zero_direction_hook = [(f"blocks.{cfg.layer}.{cfg.act_name}", get_direction_ablation_hook(
        encoder, encoder_neuron, pos
    ))]

    logprobs_active = []
    logprobs_inactive = []
    for prompt in prompts:
        logits_active = model(prompt, return_type="logits")[0, pos]
        with model.hooks(zero_direction_hook):
            logits_inactive = model(prompt, return_type="logits")[0, pos]

        logprobs_active_current = logits_active.log_softmax(dim=-1)
        logprobs_inactive_current = logits_inactive.log_softmax(dim=-1)
        logprobs_active.append(logprobs_active_current)
        logprobs_inactive.append(logprobs_inactive_current)
    logprobs_active = torch.stack(logprobs_active).mean(0)
    logprobs_inactive = torch.stack(logprobs_inactive).mean(0)
    print(logprobs_active[answer_token], logprobs_inactive[answer_token])

    boosts = (logprobs_active - logprobs_inactive)
    boosts[logprobs_active < -9] = 0
    boosts[all_ignore] = 0
    top_boosts, top_tokens = torch.topk(boosts, 15)
    non_zero_boosts = top_boosts != 0
    top_deboosts, top_deboosted_tokens = torch.topk(boosts, 15, largest=False)
    non_zero_deboosts = top_deboosts != 0
    boosted_tokens = (
        model.to_str_tokens(top_tokens[non_zero_boosts]),
        top_boosts[non_zero_boosts].tolist(),
    )
    deboosted_tokens = (
        model.to_str_tokens(top_deboosted_tokens[non_zero_deboosts]),
        top_deboosts[non_zero_deboosts].tolist(),
    )
    logging.info(f"Top boosted: {boosted_tokens}")
    logging.info(f"Top deboosted: {deboosted_tokens}")

get_direction_logit_and_logprob_boost(activating_test_prompts[:100], second_encoder, direction, model, rare_tokens, second_encoder_cfg, pos=-2)

(INFO) 09:15:52: Top boosted: (['!".', '!"', '".', '."'], [1.1426973342895508, 0.9855108261108398, 0.8336892127990723, 0.3870839476585388])
(INFO) 09:15:52: Top deboosted: (['.', '!', ' on', ' with', ' for', ',"', ',', ' at', ' that', ' in', ' because', ' and', ' so', ' to', ' when'], [-0.7691488265991211, -0.664771556854248, -0.6166729927062988, -0.6153430938720703, -0.6126704216003418, -0.5856924057006836, -0.5640039443969727, -0.5352458953857422, -0.5021038055419922, -0.4979672431945801, -0.4947929382324219, -0.49103641510009766, -0.46613025665283203, -0.39168882369995117, -0.3858466148376465])


tensor(-0.8140, device='cuda:0') tensor(-1.2011, device='cuda:0')


In [ ]:
# Logit boosts (weight based DLA)
def get_direction_boosted_tokens(direction, encoder: AutoEncoder, model: HookedTransformer, cfg: AutoEncoderConfig, rare_tokens: Tensor):
    token_boosts = encoder.W_dec[direction] @ model.W_out[cfg.layer] @ model.unembed.W_U
    token_boosts[rare_tokens] = 0
    return token_boosts

def print_token_boosts(boosts, tokens):
    str_tokens = model.to_str_tokens(tokens)
    boost_str = ""
    for token, boost in zip(str_tokens, boosts.tolist()):
        boost_str += f"('{token}': {boost:.2f}), "
    print(boost_str[:-2])

boosts = get_direction_boosted_tokens(direction, second_encoder, model, second_encoder_cfg, rare_tokens)
top_boosts, top_tokens = torch.topk(boosts, 25)
print_token_boosts(top_boosts, top_tokens)
top_boosts, top_tokens = torch.topk(boosts, 25, largest=False)
print_token_boosts(top_boosts, top_tokens)

('!".': 0.91), ('!"': 0.88), ('".': 0.81), ('?".': 0.65), ('."': 0.64), ('"': 0.58), (' I': 0.53), ('?"': 0.51), (' ok': 0.44), (' my': 0.43), (' you': 0.42), (' we': 0.40), (' -': 0.38), (' your': 0.38), (' here': 0.38), ('�': 0.37), (' mum': 0.36), ('heart': 0.35), (' yours': 0.35), (' please': 0.35), (' then': 0.34), (' today': 0.34), (' looks': 0.33), (' our': 0.33), (' yourself': 0.33)
(' She': -0.19), (' herself': -0.17), (' her': -0.14), (' train': -0.13), (' yacht': -0.13), ('ule': -0.13), (' peanut': -0.13), (' bubble': -0.13), (' muff': -0.13), ('ooter': -0.12), (' rode': -0.12), (' volcano': -0.12), ('erry': -0.11), (' Bun': -0.11), (' elevator': -0.11), ('affle': -0.11), (' whole': -0.11), ('agged': -0.11), (' statue': -0.11), (' porch': -0.11), (' ch': -0.10), (' jet': -0.10), (' drank': -0.10), (' He': -0.10), (' sprayed': -0.10)


## Additional ablations

In [ ]:
# Does ablating the feature increase loss when the feature is active?

# On positions which close quotation
loss_increases = []
for prompt in tqdm(activating_test_prompts):
    tokens = model.to_tokens(prompt)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, second_encoder, model, direction, second_encoder_cfg, pos=pos)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))
px.histogram(loss_increases, width=700, title=f"Loss increase for removing direction {direction} on closing quotation prompts")

  0%|          | 0/695 [00:00<?, ?it/s]

0.25335989884576565 0.3038651343876636


In [ ]:
test_prompt_activations = []
for prompt in tqdm(activating_test_prompts):
    act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-2, direction]
    test_prompt_activations.append(act.item())

  0%|          | 0/695 [00:00<?, ?it/s]

In [ ]:
# Histogram of activation x loss increases to check if high loss increase on a prompt just comes from the direction being very active
fig = px.scatter(x=test_prompt_activations, y=loss_increases, width=700, title="Loss increase vs activation for closing quotation prompts")
fig.update_layout({
    "xaxis_title": "Activation",
    "yaxis_title": "Loss increase"
})

In [ ]:
## Look at high loss increase prompts
# 15796: always after question in prior sentence, usually end in object (e.g. toy, mistake, door, hive)

top_loss_increases, top_loss_increase_prompt_indices = torch.topk(torch.tensor(loss_increases), 10)

def print_examples(prompts: list[str]):
    for prompt in prompts:
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

print_examples([activating_test_prompts[i] for i in top_loss_increase_prompt_indices])

In [ ]:
# In general on max activating prompts, ablating globally
loss_increases = []
max_activating_prompts, max_activating_token_indices = get_top_activating_examples_for_direction(prompts, direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, k=100)
for prompt, index in zip(max_activating_prompts, max_activating_token_indices.tolist()):
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, direction, second_encoder_cfg)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

0.006354030966758728 0.00835937654341787


In [ ]:
# In general on max activating prompts, ablating one activating position at a time
loss_increases = []
max_activating_prompts, max_activating_token_indices = get_top_activating_examples_for_direction(prompts, direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, k=100)
threshold = second_encoder_max_activations[:, direction].max() * 0.1

for prompt, index in tqdm(zip(max_activating_prompts, max_activating_token_indices.tolist()), total=len(max_activating_prompts)):
    acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[:, direction]
    # No loss for last position, exclude
    active_positions = torch.argwhere(acts[:-1] > threshold).flatten().tolist()
    for position in active_positions:
        original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, direction, second_encoder_cfg, pos=position)
        loss_increase = ablated_loss - original_loss
        loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

  0%|          | 0/100 [00:00<?, ?it/s]

0.10963073582908171 0.4840503690021192


## Check for directions with positive DLA


In [ ]:
activation_scaled_direction_dla = (all_acts.mean(0).unsqueeze(1) * second_encoder.W_dec) @ model.W_out[second_encoder_cfg.layer] @ model.unembed.W_U[:, answer_token]
top_acts, top_dirs = torch.topk(activation_scaled_direction_dla, 5)
print(top_acts, top_dirs)

torch.Size([16384])
tensor([1.4160, 0.9038, 0.1925, 0.1394, 0.1394], device='cuda:0') tensor([15796,  8093, 10593, 16264,   745], device='cuda:0')


In [ ]:
direction_dla = second_encoder.W_dec @ model.W_out[second_encoder_cfg.layer] @ model.unembed.W_U[:, answer_token]
active_directions = (all_acts.mean(0) > 0.05).cpu()
print(active_directions.sum())

fig = go.Figure(data=go.Scatter(x=direction_dla.cpu()[active_directions], y=all_acts.mean(0).cpu()[active_directions], mode='markers'))
fig.update_layout(
    title="Direction DLA vs activation",
    xaxis_title='Direction \'.\"\' DLA',
    yaxis_title='Direction activation',
    width=900
)
fig.show()

tensor(154)


In [ ]:
active_directions = (all_acts.mean(0) > 0.05).cpu()
print(active_directions.sum())

fig = go.Figure(data=go.Scatter(x=activation_scaled_direction_dla.cpu()[active_directions], y=all_acts.mean(0).cpu()[active_directions], mode='markers'))
fig.update_layout(
    title="DLA and activation of active Layer 1 encoder directions",
    xaxis_title='Direction \'.\"\' DLA',
    yaxis_title='Direction activation',
    width=900
)
fig.show()

tensor(154)
